In [1]:
!pip install azure-storage-blob

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
import numpy as np
import json
import requests 
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from io import BytesIO, StringIO

# Azure Functions
def azure_upload_blob(connect_str, container_name, blob_name, data):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    blob_client.upload_blob(data, overwrite=True)
    print(f"Uploaded to Azure Blob: {blob_name}")

def azure_download_blob(connect_str, container_name, blob_name):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    download_stream = blob_client.download_blob()
    return download_stream.readall()


In [3]:
URL = "https://data.cityofnewyork.us/api/views/h9gi-nx95/rows.csv?accessType=DOWNLOAD"

df_raw = pd.read_csv(URL, engine = 'pyarrow')
df_raw.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,09/11/2021,2:39,None,None,NaN,NaN,None,WHITESTONE EXPRESSWAY,20 AVENUE,None,...,Unspecified,None,None,None,4455765,Sedan,Sedan,None,None,None
1,03/26/2022,11:45,None,None,NaN,NaN,None,QUEENSBORO BRIDGE UPPER,None,None,...,None,None,None,None,4513547,Sedan,None,None,None,None
2,06/29/2022,6:55,None,None,NaN,NaN,None,THROGS NECK BRIDGE,None,None,...,Unspecified,None,None,None,4541903,Sedan,Pick-up Truck,None,None,None
3,09/11/2021,9:35,BROOKLYN,11208,40.667202,-73.866500,"(40.667202, -73.8665)",None,None,1211 LORING AVENUE,...,None,None,None,None,4456314,Sedan,None,None,None,None
4,12/14/2021,8:13,BROOKLYN,11233,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,None,...,None,None,None,None,4486609,None,None,None,None,None


In [4]:
df_raw.shape

(2082277, 29)

In [5]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2082277 entries, 0 to 2082276
Data columns (total 29 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   CRASH DATE                     object 
 1   CRASH TIME                     object 
 2   BOROUGH                        object 
 3   ZIP CODE                       object 
 4   LATITUDE                       float64
 5   LONGITUDE                      float64
 6   LOCATION                       object 
 7   ON STREET NAME                 object 
 8   CROSS STREET NAME              object 
 9   OFF STREET NAME                object 
 10  NUMBER OF PERSONS INJURED      float64
 11  NUMBER OF PERSONS KILLED       float64
 12  NUMBER OF PEDESTRIANS INJURED  int64  
 13  NUMBER OF PEDESTRIANS KILLED   int64  
 14  NUMBER OF CYCLIST INJURED      int64  
 15  NUMBER OF CYCLIST KILLED       int64  
 16  NUMBER OF MOTORIST INJURED     int64  
 17  NUMBER OF MOTORIST KILLED      int64  
 18  CO

In [6]:
# Check for null values in each column
null_values = df_raw.isnull().sum()

# Display the count of null values in each column
print(null_values)

CRASH DATE                             0
CRASH TIME                             0
BOROUGH                           647881
ZIP CODE                          648132
LATITUDE                          234094
LONGITUDE                         234094
LOCATION                          234094
ON STREET NAME                    442554
CROSS STREET NAME                 787961
OFF STREET NAME                  1732096
NUMBER OF PERSONS INJURED             18
NUMBER OF PERSONS KILLED              31
NUMBER OF PEDESTRIANS INJURED          0
NUMBER OF PEDESTRIANS KILLED           0
NUMBER OF CYCLIST INJURED              0
NUMBER OF CYCLIST KILLED               0
NUMBER OF MOTORIST INJURED             0
NUMBER OF MOTORIST KILLED              0
CONTRIBUTING FACTOR VEHICLE 1       6845
CONTRIBUTING FACTOR VEHICLE 2     323327
CONTRIBUTING FACTOR VEHICLE 3    1933393
CONTRIBUTING FACTOR VEHICLE 4    2048639
CONTRIBUTING FACTOR VEHICLE 5    2073154
COLLISION_ID                           0
VEHICLE TYPE COD

In [7]:
df_cleaned = df_raw.copy()


In [8]:
df_cleaned = df_raw.drop(columns=['CONTRIBUTING FACTOR VEHICLE 3', 'CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5', 'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5'])

df_cleaned.info()

                        

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2082277 entries, 0 to 2082276
Data columns (total 23 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   CRASH DATE                     object 
 1   CRASH TIME                     object 
 2   BOROUGH                        object 
 3   ZIP CODE                       object 
 4   LATITUDE                       float64
 5   LONGITUDE                      float64
 6   LOCATION                       object 
 7   ON STREET NAME                 object 
 8   CROSS STREET NAME              object 
 9   OFF STREET NAME                object 
 10  NUMBER OF PERSONS INJURED      float64
 11  NUMBER OF PERSONS KILLED       float64
 12  NUMBER OF PEDESTRIANS INJURED  int64  
 13  NUMBER OF PEDESTRIANS KILLED   int64  
 14  NUMBER OF CYCLIST INJURED      int64  
 15  NUMBER OF CYCLIST KILLED       int64  
 16  NUMBER OF MOTORIST INJURED     int64  
 17  NUMBER OF MOTORIST KILLED      int64  
 18  CO

In [9]:
df_cleaned.shape

(2082277, 23)

In [ ]:
connection_string_azure_storage= "DefaultEndpointsProtocol=https;AccountName=hwdata;AccountKey=DDwNKkt1AiZy0bLjtgJmfaZvIRGeXnjnMk57wIPXwGBT5IfyRCrIyzSOyz2nXwUuOYP6BYYtwNVQ+AStL4p/8Q==;EndpointSuffix=core.windows.net"
container_azure = 'motorvehiclecrash'
blob_name = "motorvehiclecrash.csv"

# Convert DataFrame to CSV
output = StringIO()
df_raw.to_csv(output, index=False)
data = output.getvalue()
output.close()

# Create the BlobServiceClient object
blob_service_client = BlobServiceClient.from_connection_string(connection_string_azure_storage)

# Get a blob client using the container name and blob name
blob_client = blob_service_client.get_blob_client(container=container_azure, blob=blob_name)

# Upload the CSV data
blob_client.upload_blob(data, overwrite=True)

print(f"Uploaded {blob_name} to Azure Blob Storage in container {container_azure}.")